In [61]:
from numpy import matrix, array
from numpy.linalg import pinv, inv
from collections import namedtuple
from numpy import transpose as T


def curve(X=[0,50,100],Y=[0,10,0]):
    x0,x1,x2 = X
    y0,y1,y2 = Y
    A = 1/(x1-x0)
    B = 1/(x2-x1)
    A_ = y1-y0
    B_ = y2-y1
    a = matrix(([2*A, A , 0], [A, 2*(A+B), B], [0, B, 2*B]))
    b_ = matrix(((3*A_*A**2), (3*(A_*A**2 + B_*B**2)), (3*(B_*B**2))))
    b = T(b_)
    a_inv = inv(a)
    k = a_inv * b
    k = [float(K) for K in k]
    k0, k1, k2 = k
    cx0 = x0 + x1/2
    cy0 = (x2-x0)*float(k0)/2.25
    cx2 = x2 + x1/2
    cy2 = (x0-x2)*float(k2)/2.25
    a__ = k0*(x1-x0)-(y1-y0)
    b__ = -k1*(x1-x0)+(y1-y0)
    K0 = (y1-y0)/(x1-x0) + a__/(x1-x0)
    a__ = k1*(x2-x1)-(y2-y1)
    b__ = -k2*(x2-x1)+(y2-y1)
    K2 = (y2-y1)/(x2-x1) - b__/(x2-x1)
    
#     K0 = (cy0-y0)/(cx0-x0)
    K1 = (cy2-cy0)/(cx2-cx0)
#     K2 = (y2-cy2)/(x2-cx2)
    Slopes = namedtuple('Slopes','k0 k1 k2')
    slopes = Slopes(k0,k1,k2)
    alt_slopes = Slopes(K0,K1,K2)
    print(alt_slopes)
    return slopes

def control_points(X,Y,K):
    k0, k1, k2 = K
    x0,x1,x2 = X
    y0,y1,y2 = Y
    X_M = T(matrix(array([x0,x1,x2,y0,y1,y2,y0,y2])))
    E = matrix(((1,0,0,0,0,0,0,0),
                (1/8,3/8,3/8,1/8,0,0,0,0),
                (0,0,0,1,0,0,0,0),
                (0,0,0,0,1,0,0,0),
                (0,0,0,0,1/8,3/8,3/8,1/8),
                (0,0,0,0,0,0,0,1),
                (k0,-k0,0,0,0,1,0,0),
                (0,0,-k2,k2,0,0,1,0)))
    C = inv(E)*X_M
    cx0,cx1,cx2,cx3,cy0,cy1,cy2,cy3 = C
    Coords = namedtuple('Coords','C0 C1 C2 C3')
    coords = Coords((close_(cx0),close_(cy0)),
                    (close_(cx1),close_(cy1)),
                    (close_(cx2),close_(cy2)),
                    (close_(cx3),close_(cy3)))
    return coords

def close_(num):
    num= round(float(num)*10**11)/10**11
    return num

def arb_spline(X=[0,0,0],Y=[0,0,0],K=[0,0,0],rot_angle=0,sag=0,
               fill='none', stroke='black', stroke_width='2',
               DEBUG=False):
    """Returns the information necessary to draw a spline through three points
    """
    x0,x1,x2 = X
    y0,y1,y2 = Y
    k0,k1,k2 = K
    print((X,Y,K))
    angle = rot_angle
    Sag_alt = y1 - (y2-y0)/(x2-x0)*(x1-x0)+y0
    if sag ==0 and Sag_alt!=0:
        sag = Sag_alt
    if sag!=0:
        CP = control_points(X,Y,K)
        print(CP)
        cx0 = CP.C1[0]
        cy0 = CP.C1[1]
        cx2 = CP.C2[0]
        cy2 = CP.C2[1]
    else:
        # Control Points
        cx0 = x0 + x1/2
        cy0 = (x2-x0)*float(k0)/2.25
        cx2 = x2 - x1/2
        cy2 = (x0-x2)*float(k2)/2.25
    
    p0 = [x0,y0]
    p1 = [x1,y1]
    p2 = [x2,y2]
#     if angle !=0:
#         p1 = rotate(p1,p0,angle,DEBUG=DEBUG)
#         p2 = rotate(p2,p0,angle,DEBUG=DEBUG)
#         if DEBUG:
#             print('\ncx2,cy2 = {},{}'.format(cx2,cy2))
#         cx0,cy0 = rotate([cx0,cy0],p0,angle,DEBUG=DEBUG)
#         cx0,cy0 = float(cx0),float(cy0)
#         cx2,cy2 = rotate([cx2,cy2],p0,angle,DEBUG=DEBUG)
#         cx2,cy2 = float(cx2),float(cy2)
    x0,y0 = p0
    x1,y1 = p1
    x1,y1 = float(x1),float(y1)
    x2,y2 = p2
    x2,y2 = float(x2),float(y2)

    dx = x2
    dy = y2
    Coords = namedtuple("Coords", "x0 y0 x1 y1 x2 y2")
    SVG_values = namedtuple("SVG_values","x0 y0 cx0 cy0 cx2 cy2 dx dy")
    coords = Coords(x0,y0,x1,y1,x2,y2)
    svg_values = SVG_values(x0, y0, cx0, cy0, cx2, cy2, dx, dy)
    style = 'fill:%s;stroke:%s;stroke-width:%spx'%(fill, stroke, stroke_width)
    svg_path_txt = '<path style="%s" d="M %f,%f C %f,%f %f,%f %f,%f"/>'%(style,x0,y0,cx0,cy0,cx2,cy2,dx,dy)
    
    Results = namedtuple("Results","coords svg_values svg_path_txt")
    results = Results(coords, svg_values, svg_path_txt)
    return results

In [62]:
import numpy as np

def inter(P0,P1,P2,P3):
    m1 = (P1[1]-P0[1])/(P1[0]-P0[0])
    m2 = (P3[1]-P2[1])/(P3[0]-P2[0])
    x = (P2[1]-P0[1] + P0[0]*m1 - P2[0]*m2)/(m1-m2)
    y = m1*(x-P0[0])+P0[1]
    print("\nm1, m2 = {}".format([m1,m2]))
    print((x,y))
    print()
    I = [x,y]
    line(P0,I)
    line(P3,I)
    t_1 = np.arctan2(m1,1)+np.pi
    t_2 = np.arctan2(m2,1)
    print()
    print(np.degrees(t_1),np.degrees(t_2))
    t_b = (t_2-t_1)/2+t_1
    m_b = np.tan(t_b)
    print("Bisector slope m_b = {}".format(m_b))
    print()
    x_b = (P0[1]-y)*1/m_b+x
    B = [x_b,P0[1]]
    line(I,B)
    
def mid(x0,y0,x1,y1):
    x = (x0+x1)/2
    y = (y0+y1)/2
    print('<circle cx="%g" cy="%g" r="4" stroke="black" stroke-width="3" fill="none" />'%(x,y))
    return(x,y)

def line(p0,p1):
    x0,y0 = p0
    x1,y1 = p1
    print('<line x1="%f" y1="%f" x2="%f" y2="%f" style="stroke:rgb(255,0,0);stroke-width:1" />'%(x0,y0,x1,y1))

def points(p0,p1,p2,p3):
    print(('<path\n'
           '    style="fill:none;fill-rule:evenodd;stroke'
           ':#000000;stroke-width:1px;stroke-linecap:butt;'
           'stroke-linejoin:miter;stroke-opacity:1"'))
    print('d="M {} C {} {} {}"'.format(p0,p1,p2,p3))
    print('inkscape:connector-curvature="0" />')
    line(p0,p1)
    line(p1,p2)
    line(p2,p3)
    line(p0,p3)
    x0,y0 = p0
    x1,y1 = p1
    x2,y2 = p2
    x3,y3 = p3
     
    M = mid(x0,y0,x3,y3)
    #print('a')
    a = mid(x0,y0,x1,y1)
#     print('b')
    b = mid(x1,y1,x2,y2)
#     print('c')
    c = mid(x2,y2,x3,y3)
    line(a,b)
    line(b,c)
#     print('e')
    e = mid(a[0],a[1],b[0],b[1])
#     print('f')
    f = mid(c[0],c[1],b[0],b[1])
    line(e,f)
#     print('g')
    g = mid(e[0],e[1],f[0],f[1])
    line(M,b)
    k0 = m(p0,p1)
    k1 = m(e,f)
    k2 = m(p2,p3)
    print('\nk0 = {}'.format(k0))
    print('k1 = {}\n'.format(k1))
    yM = q(M[0],x0,g[0],y0,g[1],k0,k1)
    qM = [M[0],yM]
    line(M,qM)
    print('\n{}\n'.format(m(M,g)))
    print('<circle cx="%f" cy="%f" r="4" stroke="black" stroke-width="3" fill="none" />'%(qM[0],qM[1]))
#     x_L = (M[0]+x0)/2
#     yL = q(x_L,x0,g[0],y0,g[1],k0,k1)
#     qL = [x_L,yL]
#     print('<circle cx="%f" cy="%f" r="4" stroke="black" stroke-width="3" fill="none" />'%(qL[0],qL[1]))
    inter(p0,p1,p2,p3)
    Points = namedtuple("Points",'X Y K')
    points = Points((x0,g[0],x3),(y0,g[1],y3),(k0,k1,k2))
    print()
    print((p0,p1,p2,p3))
    return points

def dist(x0,y0,x1,y1):
    return ((x1-x0)**2+(y1-y0)**2)**0.5

def m(P0,P1):
    slope = (P1[1]-P0[1])/(P1[0]-P0[0]) if P1[0]-P0[0]!=0 else 0
    return slope

def q_(x,x0,x1,y0,y1, k0, k1):
    "Returns the slope on a cubic spline at x"
    t = (x-x0)/(x1-x0)
    a = k0*(x1-x0)-(y1-y0)
    b = -k1*(x1-x0)+(y1-y0)
    q__val = (y1-y0)/(x1-x0)+(1-2*t)*(a*(1-t)+b*t)/(x1-x0)+t*(1-t)*((b-a)/(x1-x0))
    return q__val

def q(x,x0,x1,y0,y1, k0, k1):
    "Returns y coordinate at x between two points on a cubic spline"
    t = (x-x0)/(x1-x0)
    a = k0*(x1-x0)-(y1-y0)
    b = -k1*(x1-x0)+(y1-y0)
    q_val = (1-t)*y0+t*y1+t*(1-t)*(a*(1-t)+b*t)
    return q_val


In [63]:
p = points([100,450],[250,100],[350,100],[500,450])
p

<path
    style="fill:none;fill-rule:evenodd;stroke:#000000;stroke-width:1px;stroke-linecap:butt;stroke-linejoin:miter;stroke-opacity:1"
d="M [100, 450] C [250, 100] [350, 100] [500, 450]"
inkscape:connector-curvature="0" />
<line x1="100.000000" y1="450.000000" x2="250.000000" y2="100.000000" style="stroke:rgb(255,0,0);stroke-width:1" />
<line x1="250.000000" y1="100.000000" x2="350.000000" y2="100.000000" style="stroke:rgb(255,0,0);stroke-width:1" />
<line x1="350.000000" y1="100.000000" x2="500.000000" y2="450.000000" style="stroke:rgb(255,0,0);stroke-width:1" />
<line x1="100.000000" y1="450.000000" x2="500.000000" y2="450.000000" style="stroke:rgb(255,0,0);stroke-width:1" />
<circle cx="300" cy="450" r="4" stroke="black" stroke-width="3" fill="none" />
<circle cx="175" cy="275" r="4" stroke="black" stroke-width="3" fill="none" />
<circle cx="300" cy="100" r="4" stroke="black" stroke-width="3" fill="none" />
<circle cx="425" cy="275" r="4" stroke="black" stroke-width="3" fill="none

Points(X=(100, 300.0, 500), Y=(450, 187.5, 450), K=(-2.3333333333333335, 0.0, 2.3333333333333335))

In [64]:
control_points(p.X,p.Y,p.K)

Coords(C0=(100.0, 450.0), C1=(250.0, 100.0), C2=(350.0, 100.0), C3=(500.0, 450.0))

In [65]:
k = curve(p.X,p.Y)
k

Slopes(k0=-1.96875, k1=-2.842170943040401e-16, k2=1.9687500000000004)


Slopes(k0=-1.96875, k1=0.0, k2=1.9687500000000004)

In [66]:
p.K[0]/k.k0

1.1851851851851853

In [34]:
-5/k.k0

3.5555555555555554

In [22]:
arb_spline(p.X,p.Y,k)

((100, 300.0, 500), (450, 262.5, 450), Slopes(k0=-1.40625, k1=1.6653345369377348e-16, k2=1.4062499999999998))
Coords(C0=(100.0, 450.0), C1=(277.77777777778, 200.0), C2=(322.22222222222, 200.0), C3=(500.0, 450.0))


Results(coords=Coords(x0=100, y0=450, x1=300.0, y1=262.5, x2=500.0, y2=450.0), svg_values=SVG_values(x0=100, y0=450, cx0=277.77777777778, cy0=200.0, cx2=322.22222222222, cy2=200.0, dx=500.0, dy=450.0), svg_path_txt='<path style="fill:none;stroke:black;stroke-width:2px" d="m 100.000000,450.000000 c 277.777778,200.000000 322.222222,200.000000 500.000000,450.000000"/>')